切回PPT，xxdl-01-01，我们在第一讲中就给大家介绍了DL套路，刚刚我们使用了 gluon 提供的 API 来构造数据集，是两个方法，大家还记得吗？两个关键词

ArrayDataset、DataLoader

这个套路的第2步是构建网络模型，gluon 也为我们提供了 API，我们一起来看下

## xxdl-01-03，PPT.13

在模型构建方面，gluon 能帮助我们做什么呢？
1. 模型参数，之前我们是手动定义，初始化参数，要指定weight的个数和初始值（虽然我们用的是0，实际上通常不是哦），有了 gluon，妈妈再也不用担心我的参数初始化问题了； 
2. 还有更重要的，也是更强大的，就是层，输入数据之后，Sequential会依次执行每一层，并将前一层的输出，作为输入提供给后面的层

Sequential() 定义一个空的模型

gluon.nn.Dense(1) 唯一必须定义的参数就是输出节点的个数，我们这里需要的是一个输出

net.initialize() 默认随机初始化方法，需要说明的是 initialize 后，bias 的值就可以看到了，但是 weight 的值要在训练时才能看到？大家想下这是为什么？

准备好了数据集，构建好了网络模型，下一步是什么？有奖竞猜

很好，是训练

## xxdl-01-03 PPT.14

gluon.Trainer Optimizer

trainer.step(batch_size) 更新模型

## 切换到 Jupyter

In [1]:
from mxnet import gluon, nd

In [2]:
net = gluon.nn.Sequential()
net

Sequential(

)

In [3]:
type(net)

mxnet.gluon.nn.basic_layers.Sequential

In [4]:
net.add(gluon.nn.Dense(1))

In [5]:
net

Sequential(
  (0): Dense(1, linear)
)

In [6]:
dense = net[0]
dense

Dense(1, linear)

输入 dense. tab 简单讲解下

In [7]:
dense.params

dense0_ (
  Parameter dense0_weight (shape=(1, 0), dtype=<class 'numpy.float32'>)
  Parameter dense0_bias (shape=(1,), dtype=<class 'numpy.float32'>)
)

In [8]:
dense.bias

Parameter dense0_bias (shape=(1,), dtype=<class 'numpy.float32'>)

In [9]:
net.initialize()

说明 只有在 net.initialize() 之后，dense.bias.data() 才有值

In [10]:
dense.bias.data()


[ 0.]
<NDArray 1 @cpu(0)>

构造数据集

In [11]:
from mxnet import nd, autograd, gluon

In [12]:
true_w = nd.array([1, 30.6])
true_b = nd.array([-.2])
true_w, true_b

(
 [  1.          30.60000038]
 <NDArray 2 @cpu(0)>, 
 [-0.2]
 <NDArray 1 @cpu(0)>)

In [13]:
def getDataset():
    xs = nd.random_normal(shape=(1000,2))
    ys = nd.dot(xs, true_w) + true_b
    ys += 0.01 * nd.random_normal(shape=ys.shape)
    return xs, ys
xs, ys = getDataset()
dataset = gluon.data.ArrayDataset(xs, ys)
data_iter = gluon.data.DataLoader(dataset, batch_size=10,shuffle=True)

In [14]:
square_loss = gluon.loss.L2Loss()

In [15]:
eta = 0.1

In [16]:
trainer = gluon.Trainer(dense.params, 'sgd', {'learning_rate': eta})

In [17]:
id(dense.params)

4652757960

In [18]:
id(net.collect_params())

4652760032

In [19]:
epochs = 50
for e in range(epochs):
    for x, y in data_iter:
        with autograd.record():
            yhat = net(x)
            loss = square_loss(yhat, y)
        loss.backward()
        trainer.step(10)

In [20]:
dense.weight.data()


[[  0.99898815  30.59985542]]
<NDArray 1x2 @cpu(0)>

In [21]:
true_w


[  1.          30.60000038]
<NDArray 2 @cpu(0)>

In [22]:
dense.bias.data()


[-0.19859619]
<NDArray 1 @cpu(0)>

In [23]:
true_b


[-0.2]
<NDArray 1 @cpu(0)>

只有在训练后 weight.data() 才有数据

我们用 gluon 再次实现了线性回归，今天的主题是 gluon，那么，就再多介绍下 Gluon

## 切换到 PPT.15